##### Copyright 2022 The TensorFlow Authors.

In [19]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tensorflow Lite Gesture Classification Example Adding Metadata Script


This guide shows how you can go about adding the metadata into Tensorflow Lite model.

Run all steps in-order. At the end, `model_metadata.tflite` file will be downloaded.

In [ ]:
# Install the TensorFlow Lite Support Pypi package.
!pip install tflite-support-nightly

In [21]:
# Import the required packages.
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import metadata_info
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata_schema_py_generated as _metadata_fb

from google.colab import files

In [ ]:
# Upload your model and labels here.
files.upload()

In [23]:
# Declare the resource paths
_MODEL_PATH = "model.tflite"
_LABEL_PATH = "labels.txt"
_SAVE_TO_PATH = "model_metadata.tflite"

In [27]:
# Format the inline label file to 8 lines label file.
search_text = ","
replace_text = "\n"
  
# Opening our label file in read only
with open(_LABEL_PATH, 'r') as file:
  
    # Reading the content of the file
    data = file.read()
  
    # Searching and replacing the text
    data = data.replace(search_text, replace_text)
  
# Opening our label file in write only
with open(_LABEL_PATH, 'w') as file:
  
    # Writing the replaced data in our label file
    file.write(data)

In [ ]:
model_buffer = writer_utils.load_file(_MODEL_PATH)

# Create general model information.
general_md = metadata_info.GeneralMd(
    name="Guesture Classifier",
    version="v1",
    description=("Identify the most prominent gesture in the image from a "
                 "known set of categories."),
    author="TensorFlow Lite",
    licenses="Apache License. Version 2.0")

# Create input tensor information.
input_md = metadata_info.InputImageTensorMd(
    name="input image",
    description=("Input image to be classified. The expected image is "
                 "224 x 224, with three channels (red, blue, and green) per "
                 "pixel. Each element in the tensor is a value between min and "
                 "max, where (per-channel) min is [-1] and max is [1]."),
    norm_mean=[127.5],
    norm_std=[127.5],
    color_space_type=_metadata_fb.ColorSpaceType.RGB,
    tensor_type=writer_utils.get_input_tensor_types(model_buffer)[0])

# Create output tensor information.
output_md = metadata_info.ClassificationTensorMd(
    name="probability",
    description="Probabilities of the 8 labels respectively.",
    label_files=[
        metadata_info.LabelFileMd(file_path=_LABEL_PATH, locale="en")
    ],
    tensor_type=writer_utils.get_output_tensor_types(model_buffer)[0])

ImageClassifierWriter = image_classifier.MetadataWriter
# Create the metadata writer.
writer = ImageClassifierWriter.create_from_metadata_info(
    model_buffer, general_md, input_md, output_md)

# Verify the metadata generated by metadata writer.
print(writer.get_metadata_json())

# Populate the metadata into the model.
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

In [29]:
# Download the model after add metadata.
files.download(_SAVE_TO_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>